In [ ]:
# First, install deepchem
!pip install --pre deepchem



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 879.7/879.7 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 44.9 MB/s eta 0:00:00


In [ ]:
import deepchem
deepchem.__version__

'2.7.2.dev'

In [ ]:
from rdkit import Chem

We want to install NVIDIA's Apex tool, for the training pipeline used by `simple-transformers` and Weights and Biases. This package enables us to use 16-bit training, mixed precision, and distributed training without any changes to our code. Generally GPUs are good at doing 32-bit(single precision) math, not at 16-bit(half) nor 64-bit(double precision). Therefore traditionally deep learning model trainings are done in 32-bit. By switching to 16-bit, we’ll be using half the memory and theoretically less computation at the expense of the available number range and precision. However, pure 16-bit training creates a lot of problems for us (imprecise weight updates, gradient underflow and overflow). **Mixed precision training, with Apex, alleviates these problems**.

We will be installing `simple-transformers`, a library which builds ontop of HuggingFace's `transformers` package specifically for fine-tuning ChemBERTa.

In [ ]:
!git clone https://github.com/NVIDIA/apex
!cd /content/apex
!pip install -v --no-cache-dir /content/apex
!pip install transformers
!pip install simpletransformers
!pip install wandb
!cd ..

Cloning into 'apex'...
remote: Enumerating objects: 11356, done.
remote: Counting objects: 100% (3493/3493), done.
remote: Compressing objects: 100% (416/416), done.
remote: Total 11356 (delta 3225), reused 3138 (delta 3077), pack-reused 7863
Receiving objects: 100% (11356/11356), 15.36 MiB | 31.03 MiB/s, done.
Resolving deltas: 100% (7984/7984), done.
Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
Processing ./apex
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.9/807.9 kB 12.9 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 4.1 MB/s eta 0:00:00
    Creating /tmp/pip-build-env-8ivdbka1/overlay/local/bin
    changing mode of /tmp/pip-build-env-8ivdbka1/overlay/local/bin/wheel to 755
  ERROR: pip's dependency resolver does not currently take into account all the packages that are 

In [ ]:
import sys
!test -d bertviz_repo && echo "FYI: bertviz_repo directory already exists, to pull latest version uncomment this line: !rm -r bertviz_repo"
# !rm -r bertviz_repo # Uncomment if you need a clean pull from repo
!test -d bertviz_repo || git clone https://github.com/jessevig/bertviz bertviz_repo
if not 'bertviz_repo' in sys.path:
  sys.path += ['bertviz_repo']
!pip install regex

Cloning into 'bertviz_repo'...
remote: Enumerating objects: 1633, done.
remote: Counting objects: 100% (329/329), done.
remote: Compressing objects: 100% (118/118), done.
remote: Total 1633 (delta 231), reused 225 (delta 211), pack-reused 1304
Receiving objects: 100% (1633/1633), 198.37 MiB | 21.75 MiB/s, done.
Resolving deltas: 100% (1073/1073), done.


We're going to clone an auxillary repository, bert-loves-chemistry, which will enable us to use the MolNet dataloader for ChemBERTa, which automatically generates scaffold splits on any MoleculeNet dataset!

In [ ]:
!git clone https://github.com/seyonechithrananda/bert-loves-chemistry.git

Cloning into 'bert-loves-chemistry'...
remote: Enumerating objects: 1566, done.
remote: Counting objects: 100% (202/202), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 1566 (delta 96), reused 91 (delta 91), pack-reused 1364
Receiving objects: 100% (1566/1566), 55.35 MiB | 17.29 MiB/s, done.
Resolving deltas: 100% (1000/1000), done.
Updating files: 100% (122/122), done.


First, lets load the model's Byte-Pair Encoding tokenizer, and model, and setup a Huggingface pipeline for masked tokeni prediction.

In [ ]:
from transformers import AutoModelForMaskedLM, AutoTokenizer, pipeline, RobertaModel, RobertaTokenizer
from bertviz import head_view

model = AutoModelForMaskedLM.from_pretrained("seyonec/PubChem10M_SMILES_BPE_450k")
tokenizer = AutoTokenizer.from_pretrained("seyonec/PubChem10M_SMILES_BPE_450k")

fill_mask = pipeline('fill-mask', model=model, tokenizer=tokenizer)


Some weights of the model checkpoint at seyonec/PubChem10M_SMILES_BPE_450k were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


The visualization below shows the attention induced by a sample input SMILES. This view visualizes attention as lines connecting the tokens being updated (left) with the tokens being attended to (right), following the design of the figures above. Color intensity reflects the attention weight; weights close to one show as very dark lines, while weights close to zero appear as faint lines or are not visible at all. The user may highlight a particular SMILES character to see the attention from that token only. This visualization is called the attention-head view. It is based on the excellent Tensor2Tensor visualization tool, and are all generated by the [Bertviz](https://github.com/jessevig/bertviz) library.


In [ ]:
from transformers import RobertaModel, RobertaTokenizer
from bertviz import head_view

model_version = 'seyonec/PubChem10M_SMILES_BPE_450k'
model = RobertaModel.from_pretrained(model_version, output_attentions=True)
tokenizer = RobertaTokenizer.from_pretrained(model_version)

Lets use the MolNet dataloader to generate scaffold splits from the ClinTox dataset.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
dataset = pd.read_csv('/content/drive/Shareddrives/1:1 Awani Gadre/Dataset/JAK3_processed_PIC50.csv')
dataset.head()

,canonical_smiles,pIC50
0,O=C1NCc2c(-c3ccc(F)cc3F)cc(C3CCNCC3)cc2N1c1c(C...,5.000000
1,CC(C)(C)c1nc2c3ccc(F)cc3c3c(=O)[nH]ccc3c2[nH]1,8.301030
2,Cc1cccc(Cl)c1NC(=O)c1cnc(NC(=O)C2CC2)s1,7.356547
3,O=C1Nc2ccc(NC(=O)c3ccccc3)cc2/C1=C/c1ccc[nH]1,6.060481
4,O=C1Nc2ccc(C(=O)O)cc2/C1=C/c1ccc[nH]1,6.671620


In [ ]:
# from sklearn.model_selection import train_test_split

# # Assuming X is your features and y is your labels
# train_df, valid_test_df = train_test_split(dataset, test_size=0.30, random_state=42)
# test_df, valid_df = train_test_split(valid_test_df, test_size=0.50, random_state=42)


In [ ]:
import torch
from torch.nn import functional as F

from transformers import AutoModelForMaskedLM, AutoTokenizer, pipeline, RobertaModel, RobertaTokenizer
from bertviz import head_view

#model = AutoModelForMaskedLM.from_pretrained("seyonec/PubChem10M_SMILES_BPE_450k")
#tokenizer = AutoTokenizer.from_pretrained("seyonec/PubChem10M_SMILES_BPE_450k")

# 2. Modify the Model for Regression
class ChemBERTaForRegression(torch.nn.Module):
    def __init__(self):
        super(RobertaForRegression, self).__init__()
        self.roberta = model
        self.regressor = torch.nn.Linear(768, 1)  # Assuming using 'roberta-base'

    def forward(self, input_ids, attention_mask):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        pooler_output = outputs.pooler_output
        regression_output = self.regressor(pooler_output)
        return regression_output

regression_model = ChemBERTaForRegression()

In [ ]:
# train_df

In [ ]:
"""
from torch.utils.data import DataLoader, TensorDataset
texts = list(train_df['canonical_smiles'])
labels = list(train_df['pIC50'])

input_encodings = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
input_ids, attention_masks = input_encodings["input_ids"], input_encodings["attention_mask"]

labels_tensor = torch.tensor(labels).unsqueeze(-1)

# Create a DataLoader
dataset = TensorDataset(input_ids, attention_masks, labels_tensor)
train_dataloader = DataLoader(dataset, batch_size=8, shuffle=True)



texts = list(valid_df['canonical_smiles'])
labels = list(valid_df['pIC50'])

input_encodings = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
input_ids, attention_masks = input_encodings["input_ids"], input_encodings["attention_mask"]

labels_tensor = torch.tensor(labels).unsqueeze(-1)

# Create a DataLoader
dataset = TensorDataset(input_ids, attention_masks, labels_tensor)
valid_dataloader = DataLoader(dataset, batch_size=8, shuffle=True)
"""


'\nfrom torch.utils.data import DataLoader, TensorDataset\ntexts = list(train_df[\'canonical_smiles\'])\nlabels = list(train_df[\'pIC50\'])\n\ninput_encodings = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512)\ninput_ids, attention_masks = input_encodings["input_ids"], input_encodings["attention_mask"]\n\nlabels_tensor = torch.tensor(labels).unsqueeze(-1)\n\n# Create a DataLoader\ndataset = TensorDataset(input_ids, attention_masks, labels_tensor)\ntrain_dataloader = DataLoader(dataset, batch_size=8, shuffle=True)\n\n\n\ntexts = list(valid_df[\'canonical_smiles\'])\nlabels = list(valid_df[\'pIC50\'])\n\ninput_encodings = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512)\ninput_ids, attention_masks = input_encodings["input_ids"], input_encodings["attention_mask"]\n\nlabels_tensor = torch.tensor(labels).unsqueeze(-1)\n\n# Create a DataLoader\ndataset = TensorDataset(input_ids, attention_masks, labels_tensor)\nvalid_dat

In [ ]:
"""
# 2. Define Loss and Optimizer
model = RobertaForRegression().to("cuda" if torch.cuda.is_available() else "cpu")
loss_fn = torch.nn.MSELoss()
learning_rate = 0.01
num_epochs = 150


optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 3. Training Loop

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        batch = tuple(t.to("cuda" if torch.cuda.is_available() else "cpu") for t in batch)
        input_ids_batch, attention_masks_batch, labels_batch = batch
        outputs = model(input_ids_batch, attention_mask=attention_masks_batch)
        loss = loss_fn(outputs, labels_batch)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(f"Epoch: {epoch}, Loss: {total_loss / len(train_dataloader)}")

# 4. Evaluation (assuming you've split your data into train/validation sets and have a validation dataloader)
model.eval()
with torch.no_grad():
    val_loss = 0
    for batch in valid_dataloader:
        batch = tuple(t.to("cuda" if torch.cuda.is_available() else "cpu") for t in batch)
        input_ids_batch, attention_masks_batch, labels_batch = batch
        outputs = model(input_ids_batch, attention_mask=attention_masks_batch)
        loss = loss_fn(outputs, labels_batch)
        val_loss += loss.item()
    print(f"Validation Loss: {val_loss / len(valid_dataloader)}")
"""

'\n# 2. Define Loss and Optimizer\nmodel = RobertaForRegression().to("cuda" if torch.cuda.is_available() else "cpu")\nloss_fn = torch.nn.MSELoss()\nlearning_rate = 0.01\nnum_epochs = 150\n\n\noptimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)\n\n# 3. Training Loop\n\nfor epoch in range(num_epochs):\n    model.train()\n    total_loss = 0\n    for batch in train_dataloader:\n        batch = tuple(t.to("cuda" if torch.cuda.is_available() else "cpu") for t in batch)\n        input_ids_batch, attention_masks_batch, labels_batch = batch\n        outputs = model(input_ids_batch, attention_mask=attention_masks_batch)\n        loss = loss_fn(outputs, labels_batch)\n\n        optimizer.zero_grad()\n        loss.backward()\n        optimizer.step()\n\n        total_loss += loss.item()\n    print(f"Epoch: {epoch}, Loss: {total_loss / len(train_dataloader)}")\n\n# 4. Evaluation (assuming you\'ve split your data into train/validation sets and have a validation dataloader)\nmodel.eval

In [ ]:
"""
from sklearn.metrics import r2_score
# 4. Evaluation
model.eval()
all_preds = []  # List to store all predictions
all_labels = [] # List to store all true labels

with torch.no_grad():
    val_loss = 0
    for batch in valid_dataloader:
        batch = tuple(t.to("cuda" if torch.cuda.is_available() else "cpu") for t in batch)
        input_ids_batch, attention_masks_batch, labels_batch = batch
        outputs = model(input_ids_batch, attention_mask=attention_masks_batch)
        loss = loss_fn(outputs, labels_batch)
        val_loss += loss.item()

        all_preds.extend(outputs.cpu().numpy())
        all_labels.extend(labels_batch.cpu().numpy())

    r2_val = r2_score(all_labels, all_preds)
    print(f"Validation Loss: {val_loss / len(valid_dataloader)}")
    print(f"Validation R2 score: {r2_val}")
"""


'\nfrom sklearn.metrics import r2_score\n# 4. Evaluation\nmodel.eval()\nall_preds = []  # List to store all predictions\nall_labels = [] # List to store all true labels\n\nwith torch.no_grad():\n    val_loss = 0\n    for batch in valid_dataloader:\n        batch = tuple(t.to("cuda" if torch.cuda.is_available() else "cpu") for t in batch)\n        input_ids_batch, attention_masks_batch, labels_batch = batch\n        outputs = model(input_ids_batch, attention_mask=attention_masks_batch)\n        loss = loss_fn(outputs, labels_batch)\n        val_loss += loss.item()\n\n        all_preds.extend(outputs.cpu().numpy())\n        all_labels.extend(labels_batch.cpu().numpy())\n\n    r2_val = r2_score(all_labels, all_preds)\n    print(f"Validation Loss: {val_loss / len(valid_dataloader)}")\n    print(f"Validation R2 score: {r2_val}")\n'

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
import torch
from torch.nn import functional as F
from transformers import AutoTokenizer, RobertaModel
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np

# Assuming dataset is your data, X is your features and y is your labels
# X = dataset['canonical_smiles']
# y = dataset['pIC50']

# Initial train-test split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

train = pd.read_csv('/content/drive/Shareddrives/1:1 Awani Gadre/Dataset/train.csv')
test = pd.read_csv('/content/drive/Shareddrives/1:1 Awani Gadre/Dataset/test.csv')

X_train = train['canonical_smiles']
y_train = train['pIC50']
X_test = test['canonical_smiles']
y_test = test['pIC50']

# Define the Regression model
class RobertaForRegression(torch.nn.Module):
    def __init__(self, model):
        super(RobertaForRegression, self).__init__()
        self.roberta = model
        self.regressor = torch.nn.Linear(768, 1)  # Assuming using 'roberta-base'

    def forward(self, input_ids, attention_mask):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        pooler_output = outputs.pooler_output
        regression_output = self.regressor(pooler_output)
        return regression_output

# Setup tokenizer
tokenizer = AutoTokenizer.from_pretrained("seyonec/PubChem10M_SMILES_BPE_450k")

# Define loss function and learning parameters
loss_fn = torch.nn.MSELoss()
learning_rate = 0.00001
num_epochs = 100

# KFold Cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
mse_scores = []

for train_index, valid_index in kf.split(X_train):
    X_train_fold, X_valid_fold = X_train.iloc[train_index], X_train.iloc[valid_index]
    y_train_fold, y_valid_fold = y_train.iloc[train_index], y_train.iloc[valid_index]

    # Create DataLoader for training fold
    input_encodings_train = tokenizer(list(X_train_fold), return_tensors="pt", padding=True, truncation=True, max_length=512)
    train_dataset = TensorDataset(input_encodings_train["input_ids"], input_encodings_train["attention_mask"], torch.tensor(list(y_train_fold)).unsqueeze(-1))
    train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

    # Create DataLoader for validation fold
    input_encodings_valid = tokenizer(list(X_valid_fold), return_tensors="pt", padding=True, truncation=True, max_length=512)
    valid_dataset = TensorDataset(input_encodings_valid["input_ids"], input_encodings_valid["attention_mask"], torch.tensor(list(y_valid_fold)).unsqueeze(-1))
    valid_dataloader = DataLoader(valid_dataset, batch_size=64, shuffle=True)

    # Initialize model and optimizer
    model = RobertaForRegression(RobertaModel.from_pretrained("seyonec/PubChem10M_SMILES_BPE_450k")).to("cuda" if torch.cuda.is_available() else "cpu")
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        for batch in train_dataloader:
            batch = tuple(t.to("cuda" if torch.cuda.is_available() else "cpu") for t in batch)
            input_ids_batch, attention_masks_batch, labels_batch = batch
            outputs = model(input_ids_batch, attention_mask=attention_masks_batch)
            loss = loss_fn(outputs, labels_batch)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    # Validation loop
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in valid_dataloader:
            batch = tuple(t.to("cuda" if torch.cuda.is_available() else "cpu") for t in batch)
            input_ids_batch, attention_masks_batch, labels_batch = batch
            outputs = model(input_ids_batch, attention_mask=attention_masks_batch)
            all_preds.extend(outputs.cpu().numpy())
            all_labels.extend(labels_batch.cpu().numpy())

    mse_scores.append(mean_squared_error(all_labels, all_preds))

print("MSE scores from 5-fold cross validation:", mse_scores)
print("Mean Square Error: ", np.mean(mse_scores))

"""
# Now train on the entire training set
input_encodings_train = tokenizer(list(X_train), return_tensors="pt", padding=True, truncation=True, max_length=512)
train_dataset = TensorDataset(input_encodings_train["input_ids"], input_encodings_train["attention_mask"], torch.tensor(y_train).unsqueeze(-1))
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
"""



NameError: ignored